# 动态拓扑
在本教程的[上一节](../dynamic_routing)中，我们学习了如何使用[`ferry_to()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.ferry_to) API 来实现动态路由。这个功能使我们能够创建分支和循环逻辑，形成处理由运行时输入驱动的动态行为的基础。然而，当我们考虑到 LLM 的高度自主规划能力时，仅仅依靠 `ferry_to()` 提供的动态特性已经不再足够。

为了支持高度自主的 AI 应用，Bridgic 中的 Worker 协调是基于 **动态有向图 (DDG)** 构建的，其拓扑可以在运行时发生变化。这种基于 DDG 的架构在 LLM 规划的执行路径无法在编码时预先确定的场景中尤其有用。它提供了比之前描述的路由机制更大的灵活性。

## 示例：工具选择

大多数 LLM 支持工具选择和调用——这是典型代理循环中的关键步骤。
在以下示例中，我们将通过一个 *旅行规划代理* 演示工具选择的关键过程，并使用 Bridgic 的 **动态拓扑** 来实现工具调用。

**注意**：

此代码示例仅用于演示目的。它代表了完整代理循环中整体执行流程的一部分。
如果您打算在生产中使用工具调用和代理循环，请使用 Bridgic 框架提供的 [`ReActAutoma`](../../../../reference/bridgic-core/bridgic/core/agentic/#bridgic.core.agentic.ReActAutoma) 类。

运行以下 `pip` 命令以确保已安装 ['openai' 集成](../../../../reference/bridgic-llms-openai/bridgic/llms/openai/)。

```shell
pip install -U bridgic
pip install -U bridgic-llms-openai
```

### 1. 初始化

在我们开始之前，让我们初始化 OpenAI LLM 实例和运行环境。

In [3]:
import os

# Get the API base, API key and model name.
_api_key = os.environ.get("OPENAI_API_KEY")
_api_base = os.environ.get("OPENAI_API_BASE")
_model_name = os.environ.get("OPENAI_MODEL_NAME")

from bridgic.llms.openai import OpenAILlm, OpenAIConfiguration

llm = OpenAILlm(  # the llm instance
    api_base=_api_base,
    api_key=_api_key,
    configuration=OpenAIConfiguration(model=_model_name),
    timeout=20,
)

### 2. 准备工具

在旅行规划示例中，我们需要为 LLM 提供几个工具以供调用。以下代码将这些工具定义为函数。

In [5]:
# Three mock tools defined as async functions.

async def get_weather(city: str, days: int):
    """
    Get the weather forecast for the next few days in a specified city.

    Parameters
    ----------
    city : str
        The city to get the weather of, e.g. New York.
    days : int
        The number of days to get the weather forecast for.
    
    Returns
    -------
    str
        The weather forecast for the next few days in the specified city.
    """
    return f"The weather in {city} will be mostly sunny for the next {days} days."

async def get_flight_price(origin_city: str, destination_city: str):
    """
    Get the average round-trip flight price from one city to another.

    Parameters
    ----------
    origin_city : str
        The origin city of the flight.
    destination_city : str
        The destination city of the flight.
    
    Returns
    -------
    str
        The average round-trip flight price from the origin city to the destination city.
    """
    return f"The average round-trip flight from {origin_city} to {destination_city} is about $850."

async def get_hotel_price(city: str, nights: int):
    """
    Get the average price of a hotel stay in a specified city for a given number of nights.

    Parameters
    ----------
    city : str
        The city to get the hotel price of, e.g. New York.
    nights : int
        The number of nights to get the hotel price for.
    
    Returns
    -------
    str
        The average price of a hotel stay in the specified city for the given number of nights.
    """
    return f"A 3-star hotel in {city} costs about $120 per night for {nights} nights."

from bridgic.core.agentic.tool_specs import FunctionToolSpec

funcs = [get_weather, get_flight_price, get_hotel_price]
tool_list = [FunctionToolSpec.from_raw(func) for func in funcs]

在上面的代码中，定义了三个工具。每个工具的文档字符串提供了重要信息，这些信息将作为呈现给 LLM 的工具描述。每个工具被转换为一个 [`FunctionToolSpec`](../../../../reference/bridgic-core/bridgic/core/agentic/tool_specs/#bridgic.core.agentic.tool_specs.FunctionToolSpec) 实例，这三个工具被存储在 `tool_list` 变量中以供后续使用。

### 3. 编排

此演示由四个步骤组成：

1. **调用 LLM**: 将可用工具的列表传递给 LLM，并获取其 `tool_calls` 输出。
2. **动态创建 Worker**: 根据 `tool_calls` 结果动态创建 Worker。
3. **调用工具**: 让 Bridgic 框架自动调度和执行代表工具的 Worker。
4. **聚合结果**: 将执行结果组合成一个 [`ToolMessage`](../../../../reference/bridgic-core/bridgic/core/agentic/types/#bridgic.core.agentic.types.ToolMessage) 对象的列表，这些对象可以在后续处理中输入到 LLM。

我们通过子类化 [`GraphAutoma`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma) 来实现这些步骤：

In [8]:
from typing import List, Tuple, Any
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.agentic.tool_specs import ToolSpec
from bridgic.core.model.types import Message, Role, ToolCall
from bridgic.core.automa.args import From, ArgsMappingRule
from bridgic.core.agentic.types import ToolMessage

class TravelPlanner(GraphAutoma):
    @worker(is_start=True)
    async def invoke_llm(self, user_input: str, tool_list: List[ToolSpec]):
        tool_calls, _ = await llm.aselect_tool(
            messages=[
                Message.from_text(text="You are an intelligent AI assistant that can perform tasks by calling available tools.", role=Role.SYSTEM),
                Message.from_text(text=user_input, role=Role.USER),
            ], 
            tools=[tool.to_tool() for tool in tool_list], 
        )
        print(f"[invoke_llm] - LLM returns tool_calls: {tool_calls}")
        return tool_calls
    
    @worker(dependencies=["invoke_llm"])
    async def process_tool_calls(
        self,
        tool_calls: List[ToolCall],
        tool_list: List[ToolSpec],
    ):
        matched_list = self._match_tool_calls_and_tool_specs(tool_calls, tool_list)
        matched_tool_calls = []
        tool_worker_keys = []
        for tool_call, tool_spec in matched_list:
            matched_tool_calls.append(tool_call)
            tool_worker = tool_spec.create_worker()
            worker_key = f"tool_{tool_call.name}_{tool_call.id}"
            print(f"[process_tool_calls] - add worker: {worker_key}")
            self.add_worker(
                key=worker_key,
                worker=tool_worker,
            )
            self.ferry_to(worker_key, **tool_call.arguments)
            tool_worker_keys.append(worker_key)
        self.add_func_as_worker(
            key="aggregate_results",
            func=self.aggregate_results,
            dependencies=tool_worker_keys,
            args_mapping_rule=ArgsMappingRule.MERGE,
        )
        return matched_tool_calls

    async def aggregate_results(
        self, 
        tool_results: List[Any],
        tool_calls: List[ToolCall] = From("process_tool_calls"),
    ) -> List[ToolMessage]:
        print(f"[aggregate_results] - tool execution results: {tool_results}")
        tool_messages = []
        for tool_result, tool_call in zip(tool_results, tool_calls):
            tool_messages.append(ToolMessage(
                role="tool", 
                content=str(tool_result), 
                tool_call_id=tool_call.id
            ))
        # `tool_messages` may be used as the inputs of the next LLM call...
        print(f"[aggregate_results] - assembled ToolMessage list: {tool_messages}")
        return tool_messages

    def _match_tool_calls_and_tool_specs(
        self,
        tool_calls: List[ToolCall],
        tool_list: List[ToolSpec],
    ) -> List[Tuple[ToolCall, ToolSpec]]:
        matched_list: List[Tuple[ToolCall, ToolSpec]] = []
        for tool_call in tool_calls:
            for tool_spec in tool_list:
                if tool_call.name == tool_spec.tool_name:
                    matched_list.append((tool_call, tool_spec))
        return matched_list

在启动 Worker `invoke_llm` 时，调用 LLM 返回一个 [`ToolCalls`](../../../../reference/bridgic-core/bridgic/core/model/types/#bridgic.core.model.types.ToolCall) 的列表。因此，这个列表中包含的工具调用信息是动态的。

在第二个 Worker `process_tool_calls` 中，基于动态的 `tool_calls` 列表，为每个要调用的工具创建一个 Worker（通过 `tool_spec.create_worker()`），并将其添加到 DDG 中。然后，`aggregate_results` Worker 也通过 [`add_func_as_worker()`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_func_as_worker) API 动态添加到 DDG，负责聚合所有工具 Worker 的执行结果。

值得注意的是，作为 Worker 调用多个工具可以充分利用 Bridgic 框架的某些特性，例如 [并发模式](../concurrency_mode/)。在这里，这些工具能够并发执行。

### 4. 让我们运行它

让我们创建一个 `TravelPlanner` 的实例并运行它。

In [9]:
agent = TravelPlanner()
await agent.arun(
    user_input="Plan a 3-day trip to Tokyo. Check the weather forecast, estimate the flight price from San Francisco, and the hotel cost for 3 nights.",
    tool_list=tool_list,
)

[invoke_llm] - LLM returns tool_calls: [ToolCall(id='call_cLERxyz110tylRxgE4XQjaRQ', name='get_weather', arguments={'city': 'Tokyo', 'days': 3}), ToolCall(id='call_CqicPm6yZoyNksEl9HGVJEOQ', name='get_flight_price', arguments={'origin_city': 'San Francisco', 'destination_city': 'Tokyo'}), ToolCall(id='call_GscwR3pvHtzR2wTki1ndpHZp', name='get_hotel_price', arguments={'city': 'Tokyo', 'nights': 3})]
[process_tool_calls] - add worker: tool_get_weather_call_cLERxyz110tylRxgE4XQjaRQ
[process_tool_calls] - add worker: tool_get_flight_price_call_CqicPm6yZoyNksEl9HGVJEOQ
[process_tool_calls] - add worker: tool_get_hotel_price_call_GscwR3pvHtzR2wTki1ndpHZp
[aggregate_results] - tool execution results: ['The weather in Tokyo will be mostly sunny for the next 3 days.', 'The average round-trip flight from San Francisco to Tokyo is about $850.', 'A 3-star hotel in Tokyo costs about $120 per night for 3 nights.']
[aggregate_results] - assembled ToolMessage list: [{'role': 'tool', 'content': 'The we

## 我们学到了什么？

在这个 *旅行规划代理* 示例中，我们展示了如何使用 Bridgic 的 **动态拓扑** 机制为工具创建 Worker。[`GraphAutoma`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma) 类在 Bridgic 中实现为 **动态有向图 (DDG)**，以支持运行时的拓扑变化。支持动态拓扑变化的 API 包括: [`add_worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_worker)、[`add_func_as_worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_func_as_worker)、[`remove_worker`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.remove_worker) 和 [`add_dependency`](../../../../reference/bridgic-core/bridgic/core/automa/#bridgic.core.automa.GraphAutoma.add_dependency)。

您可能会注意到，在 Worker 实现代码中穿插这些 API 调用可能看起来有点杂乱。我们计划在不久的将来通过新功能来解决这个问题。